# Cleaning data

## Robust data cleaning!

### 1. **Better Type Handling**
- Uses `Int64` (nullable integer) instead of to preserve NaN values `int`
- Uses `float64` for decimal precision
- Properly handles data type conversions

### 2. **Comprehensive Data Cleaning**
- Handles multiple decimal points
- Removes currency symbols (€, $, £, etc.)
- Handles European number formats (1.234.567,89)
- Removes units (m², sqm, etc.)
- Handles various null representations

### 3. **Data Validation**
- Min/max value constraints
- Negative value handling
- Outlier detection using IQR method
- Comprehensive reporting

### 4. **Better Error Handling**
- Checks if columns exist
- Reports invalid data conversions
- Provides detailed statistics

### 5. **Production-Ready Features**
- Proper documentation
- Progress indicators
- Data quality checks
- Validation summary
- File size reporting

In [1]:
import pandas as pd
import numpy as np
import re

# ENHANCED HELPER FUNCTIONS
def clean_numeric_column(df, column, as_int=False, is_price=False, is_percentage=False,
                         allow_negative=False, min_value=None, max_value=None):
    """
    Clean and standardize numeric columns with comprehensive data validation.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the column to clean
    column : str
        Name of the column to clean
    as_int : bool
        Convert to integer type (default: False, keeps as float)
    is_price : bool
        Special handling for price/currency formats (default: False)
    is_percentage : bool
        Special handling for percentage values (default: False)
    allow_negative : bool
        Whether to allow negative values (default: False)
    min_value : float or None
        Minimum acceptable value, values below are set to NaN
    max_value : float or None
        Maximum acceptable value, values above are set to NaN

    Returns:
    --------
    pandas.DataFrame
        DataFrame with cleaned numeric column
    """

    if column not in df.columns:
        print(f"Warning: Column '{column}' not found in dataframe")
        return df

    # Store original non-null count for reporting
    original_non_null = df[column].notna().sum()

    # Create a copy to avoid SettingWithCopyWarning
    cleaned_series = df[column].copy()

    # Convert to string for cleaning, preserve NaN
    cleaned_series = cleaned_series.astype(str)

    # Replace common null representations
    null_patterns = ['nan', 'none', 'null', 'n/a', 'na', '#n/a', '#value!', '#ref!',
                     '<na>', 'missing', 'MISSING', '--', '']
    cleaned_series = cleaned_series.replace(null_patterns, np.nan, regex=False)

    # deal with multiple empty spaces
    cleaned_series = cleaned_series.str.strip()
    cleaned_series.replace('', np.nan, inplace=True)

    # Skip if all values are NaN
    if cleaned_series.isna().all():
        df[column] = np.nan
        print(f"  Warning: Column '{column}' contains only null values")
        return df

    # Handle percentage values
    if is_percentage:
        # Convert percentages like "85%", "85 %", "0.85" to decimal
        cleaned_series = cleaned_series.str.replace(r'\s*%\s*', '', regex=True)
        # Check if values are in percentage format (>1) or decimal format (<=1)
        temp_numeric = pd.to_numeric(cleaned_series, errors='coerce')
        # If most non-null values are > 1, assume they're percentages
        if (temp_numeric > 1).sum() > (temp_numeric <= 1).sum():
            temp_numeric = temp_numeric / 100
        cleaned_series = temp_numeric

    # Handle price/currency values
    elif is_price:
        # Remove currency symbols: €, $, £, ¥, CHF, USD, EUR, etc.
        cleaned_series = cleaned_series.str.replace(r'[€$£¥₹₽¢]', '', regex=True)
        cleaned_series = cleaned_series.str.replace(r'\b(USD|EUR|GBP|CHF|CAD|AUD|JPY)\b', '',
                                                   regex=True, flags=re.IGNORECASE)

        # Handle European number format: 1.234.567,89 -> 1234567.89
        # First, remove thousand separators (dots, spaces, apostrophes)
        cleaned_series = cleaned_series.str.replace(r'(?<=\d)[.\s\'\u202f\u00a0]+(?=\d{3})', '', regex=True)

        # Convert decimal comma to dot: 123,45 -> 123.45
        # But only if it's the last comma (to avoid issues with thousand separators)
        cleaned_series = cleaned_series.str.replace(r',(\d{1,2})$', r'.\1', regex=True)

        # Remove any remaining non-numeric characters except decimal point and minus
        cleaned_series = cleaned_series.str.replace(r'[^\d.\-]', '', regex=True)

    # Handle regular numeric values (areas, counts, etc.)
    else:
        # Remove units like m², m2, sqm, sq.m, cm, km, etc.
        cleaned_series = cleaned_series.str.replace(r'\s*(m[²2]|sqm?|sq\.?\s*m|cm|km|ft²?|square\s*meters?)\s*',
                                                   '', regex=True, flags=re.IGNORECASE)

        # Remove thousand separators and normalize decimal separators
        cleaned_series = cleaned_series.str.replace(r'(?<=\d)[.\s\'\u202f\u00a0]+(?=\d{3})', '', regex=True)
        cleaned_series = cleaned_series.str.replace(',', '.', regex=False)

        # Remove any remaining non-numeric characters except decimal point and minus
        cleaned_series = cleaned_series.str.replace(r'[^\d.\-]', '', regex=True)

    # Handle multiple decimal points (keep only the last one)
    def fix_multiple_decimals(val):
        if pd.isna(val) or val == '':
            return np.nan
        if isinstance(val, str) and val.count('.') > 1:
            parts = val.rsplit('.', 1)  # Split at the last decimal point
            return parts[0].replace('.', '') + '.' + parts[1]
        return val

    cleaned_series = cleaned_series.apply(fix_multiple_decimals)

    # Convert to numeric, coercing errors to NaN
    cleaned_series = pd.to_numeric(cleaned_series, errors='coerce')

    # Handle negative values
    if not allow_negative:
        negative_count = (cleaned_series < 0).sum()
        if negative_count > 0:
            print(f"  Warning: Found {negative_count} negative values in '{column}', setting to NaN")
        cleaned_series = cleaned_series.where(cleaned_series >= 0, np.nan)

    # Apply min/max value constraints
    if min_value is not None:
        out_of_range = (cleaned_series < min_value).sum()
        if out_of_range > 0:
            print(f"  Warning: Found {out_of_range} values below minimum ({min_value}) in '{column}'")
        cleaned_series = cleaned_series.where(cleaned_series >= min_value, np.nan)

    if max_value is not None:
        out_of_range = (cleaned_series > max_value).sum()
        if out_of_range > 0:
            print(f"  Warning: Found {out_of_range} values above maximum ({max_value}) in '{column}'")
        cleaned_series = cleaned_series.where(cleaned_series <= max_value, np.nan)

    # Convert to appropriate dtype
    if as_int:
        # Use Int64 (nullable integer) instead of int to preserve NaN
        cleaned_series = cleaned_series.astype('Int64')
    else:
        # Keep as float64 for decimal precision
        cleaned_series = cleaned_series.astype('float64')

    # Report cleaning results
    final_non_null = cleaned_series.notna().sum()
    nulls_created = original_non_null - final_non_null
    if nulls_created > 0:
        print(f"  Cleaned '{column}': {nulls_created} values converted to null due to invalid data")

    # Assign back to dataframe
    df[column] = cleaned_series

    return df


def validate_and_report(df, column, stats=True):
    """
    Validate and report statistics for a cleaned numeric column.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the column
    column : str
        Name of the column to validate
    stats : bool
        Whether to print detailed statistics
    """
    if column not in df.columns:
        return

    col_data = df[column]

    print(f"\n--- Statistics for '{column}' ---")
    print(f"  Total rows: {len(col_data)}")
    print(f"  Non-null values: {col_data.notna().sum()} ({col_data.notna().sum()/len(col_data)*100:.1f}%)")
    print(f"  Null values: {col_data.isna().sum()} ({col_data.isna().sum()/len(col_data)*100:.1f}%)")
    print(f"  Data type: {col_data.dtype}")

    if stats and col_data.notna().any() and (col_data.dtype.kind == 'f' or col_data.dtype.kind == 'i'):
        print(f"  Min: {col_data.min()}")
        print(f"  Max: {col_data.max()}")
        print(f"  Mean: {col_data.mean():.2f}")
        print(f"  Median: {col_data.median():.2f}")

        # Detect potential outliers using IQR method
        Q1 = col_data.quantile(0.25)
        Q3 = col_data.quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((col_data < (Q1 - 1.5 * IQR)) | (col_data > (Q3 + 1.5 * IQR))).sum()
        if outliers > 0:
            print(f"  ⚠️  Potential outliers (IQR method): {outliers} ({outliers/col_data.notna().sum()*100:.1f}%)")



This class should contain all the functions that helpls us clean the data.

In [2]:
import pandas as pd

# column names copied from models.py to use in this script for reference:
    # property_id
    # locality_name
    # postal_code
    # price - remove possible currency
    # property_type
    # number_of_rooms
    # living_area: - remove possible units m2
    # equipped_kitchen
    # furnished
    # open_fire
    # terrace_area - remove possible units m2
    # garden_area - remove possible units m2
    # number_of_facades
    # swimming_pool
    # state_of_building

# MAIN CLASS
class DataProcessing:
    def __init__(self, file_path='../data/raw/scraped_data.csv'):
        # update line of code above with local CSV file path to load data <---
        # ensure old df is cleared so a new file will truly be read (and not a cached file)
        if hasattr(self, 'df'):
            del self.df
        # auto-detect separator in CSV file
        with open(file_path, 'r', encoding='utf-8') as f:
            first_line = f.readline()
            sep = ';' if ';' in first_line else ','  # choose ';' if present, else ','
        # load full csv file
        self.df = pd.read_csv(file_path, sep=sep, dtype={"property_id": str}, low_memory=False)
        print("Detected separator:", repr(sep))
        print("\nBefore any cleaning:")
        print(self.df.dtypes,"\n")
        print(self.df.head(5))
        print("\nNumber of rows raw data loaded:", len(self.df))

    def process_data(self): # main method to process data, further methods detailed below
        self.clean_price()
        # self.filter_out_type('life sale')
        self.clean_areas()
        self.convert_yes_no_columns()
        self.clean_other_numeric_columns()
        self.remove_duplicates()
        self.remove_empty_rows()
        self.fill_missing()

    def clean_price(self): # method to clean the price column
        if 'price' in self.df.columns:
            self.df = clean_numeric_column(self.df, 'price', as_int=True, is_price=True)
            print("Cleaning price fields...")

    # def filter_out_type(self, type_of_sale): # method to remove rows with a specific property type e.g. life sale
    #     if 'type_of_sale' in self.df.columns:
    #         self.df = self.df[self.df['type_of_sale'].str.lower() != type_of_sale.lower()]
    #         print("Filtering out life sale property types...")
    #         print(f"Number of rows left after filtering out life sale property type = {len(self.df)}")

    def clean_areas(self):  # method to clean the area columns
        for col in ['total_area_sqm', 'terrace_sqm', 'garden_sqm']:
            if col in self.df.columns:
                # Remove units like 'm2', 'm²' (case-insensitive)
                self.df[col] = self.df[col].astype(str).str.replace(r'\s*m[²2]', '', regex=True)
                self.df = clean_numeric_column(self.df, col, as_int=True)
        print("Cleaning area fields...")

    def convert_yes_no_columns(self):  # method to convert yes/no to 1/0
        yes_no_map = {'yes': 1, 'y': 1, 'no': 0, 'n': 0}
        for col in ['fl_furnished', 'equipped_kitchen', 'fl_open_fire', 'fl_swimming_pool']:
            if col in self.df.columns:
                self.df[col] = (
                    self.df[col]
                    .astype(str)
                    .str.strip()
                    .str.lower()
                    .map(yes_no_map)
                    .fillna(0)
                    .astype(int)
                )
        print("Converting Yes/No columns to 1/0 integers...")

    def clean_other_numeric_columns(self): # convert other numeric columns to integers
        for col in ['number_of_rooms', 'number_of_facades']:
            if col in self.df.columns:
                self.df = clean_numeric_column(self.df, col, as_int=True)
        print("Cleaning other numeric fields...")

    def remove_duplicates(self): # method to remove duplicates based on all columns except property_id
        cols_to_check = [col for col in self.df.columns if col != 'property_id']
        # Find duplicates
        duplicates_mask = self.df.duplicated(subset=cols_to_check, keep=False)
        num_duplicates = duplicates_mask.sum()
        if num_duplicates > 0:
            print(f"\nFound {num_duplicates} duplicate row(s)")
            # print(self.df[duplicates_mask].sort_values(by=cols_to_check).head(10)) # showing first 10 duplicates
        else:
            print("\nNo duplicate rows found.")
        self.df.drop_duplicates(subset=cols_to_check, keep='first', inplace=True)
        print("Removing duplicates...")
        print(f"Number of rows left after removing duplicates = {len(self.df)}")

    def remove_empty_rows(self):  # method to remove rows where id is missing or all other fields are empty
        # Remove rows without id first
        missing_id_mask = self.df['id'].isna() | (self.df['id'].astype(str).str.strip() == '')
        num_missing = missing_id_mask.sum()
        print(f"\nFound {num_missing} row(s) with missing id")
        self.df = self.df.loc[~missing_id_mask]

        # Remove rows where ALL other fields are empty
        critical_cols = [col for col in self.df.columns if col != 'id']
        all_empty_mask = pd.Series(True, index=self.df.index)

        for col in critical_cols:
            if self.df[col].dtype in [int, float]:
                col_has_value = ~self.df[col].isna() & (self.df[col] != 0)
            else:
                col_has_value = ~self.df[col].astype(str).str.strip().eq('') & ~self.df[col].isna()
            all_empty_mask &= ~col_has_value  # Keep True only if ALL fields are empty

        num_empty_rows = all_empty_mask.sum()
        print(f"Found {num_empty_rows} row(s) where all fields are empty")
        self.df = self.df.loc[~all_empty_mask]
        print("Removing empty rows...")
        print(f"Number of rows left after removing empty rows = {len(self.df)}")

    def fill_missing(self): # method to fill missing fields except for id
        for col in self.df.columns:
            if col != 'id':
                if self.df[col].dtype.kind in ['f', 'i'] :
                    # self.df[col] = self.df[col].fillna(0)
                    pass
                else:
                    self.df[col] = self.df[col].fillna('')
        print("\nFilling missing fields...")
    def clean_zip_code(self):
        if 'zip_code' in self.df.columns:
            self.df = clean_numeric_column(self.df, 'zip_code', as_int=True)
            print("Cleaning zip code field...")

    def save_to_csv(self, output_path='../data/cleaned/cleaned_property_data.csv'): # method to create the output file, update file path <---
        self.df.to_csv(output_path, index=False)
        print("\nSaving cleaned output as csv ...")

In [3]:
dp = DataProcessing(file_path='../Bryan/sample_data_copy/properties.csv')  # adjust path
dp.process_data()

for column in dp.df.columns:
    validate_and_report(dp.df, column)

dp.save_to_csv('../Bryan/cleaned_properties.csv')

Detected separator: ','

Before any cleaning:
id                                  int64
price                             float64
property_type                      object
subproperty_type                   object
region                             object
province                           object
locality                           object
zip_code                            int64
latitude                          float64
longitude                         float64
construction_year                 float64
total_area_sqm                    float64
surface_land_sqm                  float64
nbr_frontages                     float64
nbr_bedrooms                      float64
equipped_kitchen                   object
fl_furnished                        int64
fl_open_fire                        int64
fl_terrace                          int64
terrace_sqm                       float64
fl_garden                           int64
garden_sqm                        float64
fl_swimming_pool              